In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:



ValueError: ignored

In [ ]:
! cp /content/drive/MyDrive/atmacup10_dataset.zip .
! unzip atmacup10_dataset.zip

In [7]:
!pip install japanize-matplotlib > /dev/null
!pip install transformers > /dev/null

In [8]:
import os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import japanize_matplotlib
japanize_matplotlib.japanize() 
%matplotlib inline

#import dask.dataframe as dd
import pandas_profiling as pdp
from pandas_profiling import ProfileReport # profile report を作る用
from matplotlib_venn import venn2 # venn図を作成する用
from collections import defaultdict

# transformers というライブラリに入ってる BERT を使用します
import torch
import transformers
from transformers import BertTokenizer
from tqdm import tqdm
import umap

#警告が鬱陶しい時はこれを記述
import warnings
warnings.filterwarnings('ignore')

INPUT_DIR='./'
OUTPUT_DIR = './output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed = 777
seed_everything(seed)


In [4]:
color_df = pd.read_csv('/content/color.csv')
history_df = pd.read_csv('/content/historical_person.csv')
maker_df = pd.read_csv('/content/maker.csv')
material_df = pd.read_csv('/content/material.csv')
object_df = pd.read_csv('/content/object_collection.csv')
palette_df = pd.read_csv('/content/palette.csv')
pri_maker_df = pd.read_csv('/content/principal_maker.csv')
pri_maker_occp_df = pd.read_csv('/content/principal_maker_occupation.csv')
prod_place_df = pd.read_csv('/content/production_place.csv')
tech_df = pd.read_csv('/content/technique.csv')
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

FileNotFoundError: ignored

In [5]:
train.head()

NameError: ignored

# tutorial 2

In [7]:
# https://github.com/nyk510/vivid/blob/master/vivid/utils.py
from time import time

def decorate(s: str, decoration=None):
    if decoration is None:
        decoration = '★' * 20

    return ' '.join([decoration, str(s), decoration])

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        if self.verbose is None:
            return
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [8]:
import inspect

def param_to_name(params: dict, key_sep='_', key_value_sep='=') -> str:
    """
    dictを'key=value で連結したstringに変換
    params:
    key_sep: key同士を連結する際に使う文字列
    key_value_sep: それぞれの key/value を連結するのに使う文字列
    """

    sorted_params = sorted(params.items())
    return key_sep.join(map(lambda x: key_value_sep.join(map(str, x)), sorted_params))

def cachable(function):
    # 関数の返り値をキャッシュしておく
    attr_name = '__cachefile__'
    def wrapper(*args, **kwargs):
        force = kwargs.pop('force', False)
        call_args = inspect.getcallargs(function, *args, **kwargs)
        print(call_args)
        arg_name = param_to_name(call_args)
        print('arg_name', arg_name)
        name = attr_name + arg_name
        print(name)

        use_cache = hasattr(function, name) and not force

        if use_cache:
            cache_object = getattr(function, name)
        else:
            print('run')
            cache_object = function(*args, **kwargs)
            setattr(function, name, cache_object)
        return cache_object

    return wrapper

In [9]:
@cachable
def read_csv(name):
    if '.csv' not in name:
        name = name + '.csv'

    return pd.read_csv(os.path.join(INPUT_DIR, name))

# EDA


In [10]:
!pip install ptitprince

     |████████████████████████████████| 61kB 8.6MB/s 
  Created wheel for ptitprince: filename=ptitprince-0.2.5-cp37-none-any.whl size=8429 sha256=5d163612d42db53f74efa27b1b36188418a3ed67495afab28159c4596ccd50c0
  Stored in directory: /root/.cache/pip/wheels/fb/c1/75/2ef8da161053bdf8e03f9a360f426accd4fac1ddf7f6ed0029
Successfully built ptitprince


In [11]:
from ptitprince import RainCloud
train = read_csv('train')
test = read_csv('test')

{'name': 'train'}
arg_name name=train
__cachefile__name=train
run
{'name': 'test'}
arg_name name=test
__cachefile__name=test
run


In [12]:
# わかりやすさのため１０を亭としたlog変換
train['log_likes'] = np.log10(train['likes'] + 1)

In [13]:
fig, ax = plt.subplots(figsize=(8, 5))
RainCloud(data=train, y='log_likes', orient='h', ax=ax)
ax.grid()

正規ごとのいいね吸う

1回目で作ったモデルはsubtitleの長さがよく重視されていたので、これごとにlikeの変化を見る

In [14]:
sub_title_length = train['sub_title'].str.len().rename('subtitle_length')

In [15]:
fig, ax = plt.subplots(figsize=(20,6))
RainCloud(y=train['log_likes'], x=pd.cut(sub_title_length, bins=10, duplicates='drop'), ax=ax)
ax.grid()

In [16]:
# awuisition_method
fig, ax = plt.subplots(figsize=(20, 6))
RainCloud(data=train, x='acquisition_method', y='log_likes', ax=ax)
ax.grid()

* bequest (遺贈)はゼロが少ない. loan, nationalization もたかめ
* unkown transfer はゼロ多い

## 作家の影響
* 有名な作家ならいいね数が多そう。

In [17]:
x = train.groupby('principal_maker')['likes'].agg(['std', 'mean'])

In [18]:
x.sort_values('std').dropna()

,std,mean
principal_maker,,
Paul Robert,0.000000,0.000000
Poulton,0.000000,0.000000
Ludovico Tuminello,0.000000,0.000000
Louis Surugue,0.000000,0.000000
Carlo Lasinio,0.000000,0.000000
...,...,...
Francesco Zugno,3930.099490,2793.000000
Jozef Israëls,5053.646851,1437.733333
Jan Asselijn,5421.454756,3980.500000


In [19]:
name = 'Rembrandt van Rijn' # レンブラント(夜警の人)
idx = train['principal_maker'] == name
train[idx].sort_values('likes')[['principal_maker', 'title', 'likes', 'dating_sorting_date']]

,principal_maker,title,likes,dating_sorting_date
1976,Rembrandt van Rijn,Bevrijding van Petrus uit de gevangenis,0,1635.0
1764,Rembrandt van Rijn,Jozef verklaart de dromen van de bakker en de ...,4,1616.0
2815,Rembrandt van Rijn,Heilige familie,4,1616.0
2911,Rembrandt van Rijn,Christus predikt,6,1616.0
5465,Rembrandt van Rijn,Buste van een man met tulband en baard,8,1616.0
1660,Rembrandt van Rijn,"Portrait of a man, perhaps Rembrandt's father,...",42,1730.0
1982,Rembrandt van Rijn,Laughing Young Man,130,1629.0
827,Rembrandt van Rijn,Portret van Rembrandt Harmensz. van Rijn,278,1640.0
966,Rembrandt van Rijn,Salome ontvangt het hoofd van Johannes de Doper,387,1640.0
3314,Rembrandt van Rijn,Portrait of Ephraim Bueno,906,1645.0


## たくさんの特徴量の作成

### 状態管理のためにクラスを使った実装をする

In [20]:
class AbstractBaseBlock:
    def fit(self, input_df: pd.DataFrame, y=None):
        return self.transform(input_df)

    def transform(self, input_df: pd.DataFrame) -> pd.DataFrame:
        raise NotImplementedError()

#### 内部状態が更新されない例

In [21]:
class NumericBlock(AbstractBaseBlock):
    def transform(self, input_df):
        use_columns = [
            'dating_period',
            'dating_year_early',
            'dating_year_late'
        ]

        return input_df[use_columns].copy()

class StringLengthBlock(AbstractBaseBlock):
    def __init__(self, column):
        self.column = column

    def transform(self, input_df):
        out_df = pd.DataFrame()
        out_df[self.column] = input_df[self.column].str.len()
        return out_df.add_prefix('StringLength_')

#### 内部状態が更新される Countencoding

In [22]:
def read_whole_df():
    return pd.concat([
        read_csv('train'), read_csv('test')
    ], ignore_index=True)

class CountEncodingBlock(AbstractBaseBlock):
    """CountEncodingを行なう block"""
    def __init__(self, column: str):
        self.column = column

    def fit(self, input_df, y=None):
#         vc = input_df[self.column].value_counts()
        master_df = read_whole_df()
        vc = master_df[self.column].value_counts()
        self.count_ = vc
        return self.transform(input_df)

    def transform(self, input_df):
        out_df = pd.DataFrame()
        out_df[self.column] = input_df[self.column].map(self.count_)
        return out_df.add_prefix('CE_')

class OneHotEncoding(AbstractBaseBlock):
    def __init__(self, column, min_count=30):
        self.column = column
        self.min_count = min_count

    def fit(self, input_df, y=None):
        x = input_df[self.column]
        vc = x.value_counts()
        categories = vc[vc > self.min_count].index
        self.categories_ = categories

        return self.transform(input_df)

    def transform(self, input_df):
        x = input_df[self.column]
        cat = pd.Categorical(x, categories=self.categories_)
        out_df = pd.get_dummies(cat)
        out_df.columns = out_df.columns.tolist()
        return out_df.add_prefix(f'{self.column}=')

In [23]:
block = OneHotEncoding('principal_maker', min_count=40)


In [24]:
feature_blocks = [
    *[OneHotEncoding(c, min_count=20) for c in ['title', 'description', 'long_title', #リストの中にリストを展開している
       'principal_maker', 'principal_or_first_maker', 'sub_title',
       'copyright_holder', 'more_title', 'acquisition_method',
       'acquisition_date', 'acquisition_credit_line', 'dating_presenting_date',
       'dating_sorting_date', 'dating_period', 'dating_year_early',
       'dating_year_late',]],
    *[CountEncodingBlock(c) for c in ['art_series_id', 'title', 'description', 'long_title',
       'principal_maker', 'principal_or_first_maker', 'sub_title',
       'copyright_holder', 'more_title', 'acquisition_method',
       'acquisition_date', 'acquisition_credit_line', 'dating_presenting_date',
       'dating_sorting_date', 'dating_period', 'dating_year_early',
       'dating_year_late',]],
    *[StringLengthBlock(c) for c in [
        'title', 'description', 'long_title',
       'principal_maker', 'principal_or_first_maker', 'sub_title',
    ]]
]
feature_blocks

In [25]:
def run_blocks(input_df, blocks, y=None, test=False):
    out_df = pd.DataFrame()

    print(decorate('start run blocks...'))

    with Timer(prefix='run test={}'.format(test)):
        for block in feature_blocks:
            with Timer(prefix='\t- {}'.format(str(block))):
                if not test:
                    out_i = block.fit(input_df, y=y)
                else:
                    out_i = block.transform(input_df)

            assert len(input_df) == len(out_i), block
            name = block.__class__.__name__ #block処理クラス名
            out_df = pd.concat([out_df, out_i.add_suffix(f'@{name}')], axis=1)

    return out_df

In [26]:
run_blocks(train, blocks=feature_blocks)
run_blocks(test, blocks=feature_blocks, test=True)

★★★★★★★★★★★★★★★★★★★★ start run blocks... ★★★★★★★★★★★★★★★★★★★★
	- <__main__.OneHotEncoding object at 0x7f39d5d24a50> 0.014[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24a10> 0.008[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24ad0> 0.007[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24bd0> 0.011[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24c10> 0.006[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24c90> 0.007[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24cd0> 0.003[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24d10> 0.008[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24d50> 0.003[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24c50> 0.005[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24d90> 0.004[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24dd0> 0.006[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24e10> 0.006[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24e50> 0.002[s]
	- <__main__.OneHotEncoding object at 0x7f39d5d24e90> 0.

,title=Portret van een onbekende vrouw@OneHotEncoding,title=Portret van een onbekende man@OneHotEncoding,title=Portrait of a Man@OneHotEncoding,title=Portret van een onbekende jonge vrouw@OneHotEncoding,title=Portret van een vrouw@OneHotEncoding,description=Deze foto maakt deel uit van een album.@OneHotEncoding,description=Onderdeel van Album met foto's van bezienswaardigheden en landschappen in Engeland en van kunst.@OneHotEncoding,"description=Onderdeel van Engels familiealbum met foto's van personen, reizen, cricket en kunstwerken.@OneHotEncoding",description=Deze prent maakt deel uit van een album.@OneHotEncoding,description=Onderdeel van fotoalbum overstroming Parijs en voorsteden 1910.@OneHotEncoding,"description=Onderdeel van Album met foto's en litho's van bezienswaardigheden in Noord- en Zuid-Hollandse steden, schilderijen en klederdrachten.@OneHotEncoding",description=Onderdeel van Album met cartes-de-visite en kabinetfoto's van militairen in uniform.@OneHotEncoding,"description=Onderdeel van Reisalbum met opnames van bezienswaardigheden in India, Duitsland, Zwitserland en Frankrijk.@OneHotEncoding","description=Onderdeel van Reisalbum met opnames van kunstwerken, personen en bezienswaardigheden in Spanje.@OneHotEncoding",description=Onderdeel van Reisalbum met foto's van bedrijvigheid en bezienswaardigheden op Sumatra en Java en van de reis naar en van Nederlands-Indië.@OneHotEncoding,"description=Onderdeel van Reisalbum van H. Dunlop met foto's van bezienswaardigheden in Perzië, Schotland, Duitsland, Rusland, China en Canada.@OneHotEncoding",description=Onderdeel van Reisalbum met foto's van bezienswaardigheden in Napels en omgeving.@OneHotEncoding,description=Onderdeel van Engels fotoalbum van E.A. Kettle met 42 cartes-de-visite.@OneHotEncoding,description=Onderdeel van Reisalbum met veertig foto's van Nederlands-Indië.@OneHotEncoding,principal_maker=anonymous@OneHotEncoding,principal_maker=Johann Sadeler (I)@OneHotEncoding,principal_maker=Bernard Picart@OneHotEncoding,principal_maker=Richard Tepe@OneHotEncoding,principal_maker=anoniem (Monumentenzorg)@OneHotEncoding,principal_maker=Jan Luyken@OneHotEncoding,principal_maker=Raphaël Sadeler (I)@OneHotEncoding,principal_maker=Woodbury & Page@OneHotEncoding,principal_maker=Romeyn de Hooghe@OneHotEncoding,principal_maker=Aegidius Sadeler@OneHotEncoding,principal_maker=Giorgio Sommer@OneHotEncoding,principal_maker=Willem Diepraam@OneHotEncoding,principal_maker=Jacob Olie jr.@OneHotEncoding,principal_maker=Geldolph Adriaan Kessler@OneHotEncoding,principal_maker=Abraham Rademaker@OneHotEncoding,principal_maker=L. Heldring@OneHotEncoding,principal_maker=Hermanus Jan Hendrik van Rijkelijkhuysen@OneHotEncoding,principal_maker=Jan Harmensz. Muller@OneHotEncoding,principal_maker=Crispijn van de Passe (I)@OneHotEncoding,principal_maker=Johanna Margaretha Piek@OneHotEncoding,principal_maker=Cornelis Galle (I)@OneHotEncoding,...,dating_year_late=1638.0@OneHotEncoding,dating_year_late=1707.0@OneHotEncoding,dating_year_late=1630.0@OneHotEncoding,dating_year_late=1721.0@OneHotEncoding,dating_year_late=1730.0@OneHotEncoding,dating_year_late=1592.0@OneHotEncoding,dating_year_late=1651.0@OneHotEncoding,dating_year_late=1844.0@OneHotEncoding,dating_year_late=1679.0@OneHotEncoding,dating_year_late=1788.0@OneHotEncoding,dating_year_late=1612.0@OneHotEncoding,dating_year_late=1603.0@OneHotEncoding,dating_year_late=1631.0@OneHotEncoding,dating_year_late=1653.0@OneHotEncoding,dating_year_late=1548.0@OneHotEncoding,dating_year_late=1643.0@OneHotEncoding,dating_year_late=1727.0@OneHotEncoding,CE_art_series_id@CountEncodingBlock,CE_title@CountEncodingBlock,CE_description@CountEncodingBlock,CE_long_title@CountEncodingBlock,CE_principal_maker@CountEncodingBlock,CE_principal_or_first_maker@CountEncodingBlock,CE_sub_title@CountEncodingBlock,CE_copyright_holder@CountEncodingBlock,CE_more_title@CountEncodingBlock,CE_acquisition_method@CountEncodingBlock,CE_acquisition_date@CountEncodingBlock,CE_acqu

## 外部のテーブル情報を使う

### BERT Embedding

In [27]:
class BertSequenceVectorizer:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_name = 'bert-base-uncased'
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128


    def vectorize(self, sentence : str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy() # 0番目は [CLS] token, 768 dim の文章特徴量
        else:
            return seq_out[0][0].detach().numpy()

In [28]:
def create_input_array_umap(input_df, column):
    """umap用の配列作成"""
    features = []
    for array in input_df[column]:
        features.append(array)
    return pd.DataFrame(features).values.astype(np.float32)
    
class BertEmbeddingBlock(AbstractBaseBlock):
    def __init__(self, column, umap_components=None):
        self.column = column
        self.umap_components = umap_components

    def umap(self, df):
        mapper = umap.UMAP(n_components=self.umap_components)
        features = create_input_array_umap(df, 'description_feature')
        embedding = mapper.fit_transform(features)
        # sns.scatterplot(x=embedding[:,0], y=embedding[:,1], hue=train['log_likes'], ) #hueの値で色付けして表示
        # plt.show()        
        return pd.DataFrame(embedding)

    def transform(self, input_df):
        BSV = BertSequenceVectorizer() # インスタンス化します
        #train = pd.read_csv()
        out_df = pd.DataFrame()
        out_df['description'] = input_df[self.column].fillna("NaN") # null は代わりのテキストで埋めます
        out_df['description_feature'] = out_df['description'].apply(lambda x: BSV.vectorize(x))
        out_df = out_df.drop('description', axis=1)

        # umapで圧縮する場合
        if self.umap_components:
            out_df = self.umap(out_df)
            return out_df.add_prefix('BertEmbeddingBlock_')

        # BERTのをそのまま使う場合
        out_df = out_df['description_feature'].apply(pd.Series) #リストの各要素を行に展開
        return out_df.add_prefix('BertEmbeddingBlock_')

In [29]:
# 長いので省略
# block = BertEmbeddingBlock('description')
# assert len(block.fit(train)) == len(train)
# assert block.transform(test).equals(block.transform(test))

#### UMAP可視化
* PCAとかにも応用可能
* https://www.guruguru.science/competitions/16/discussions/a4894479-f006-4ce3-92f3-287d87844ec7/

In [30]:

# def create_input_array_umap(input_df):
#     features = []
#     for array in input_df['description_feature']:
#         features.append(array)
#     return pd.DataFrame(features).values.astype(np.float32)

# import umap
# mapper = umap.UMAP(n_components=50)
# features = create_input_array_umap(df)
# embedding = mapper.fit_transform(features)
# embedding.shape
# sns.scatterplot(x=embedding[:,0], y=embedding[:,1], hue=train['log_likes'], ) #hueの値で色付けして表示
# plt.show()

In [31]:
# def create_input_array(input_df):
#     features = []
#     for array in input_df['description_feature']:
#         features.append(array)
#     return pd.DataFrame(features).values.astype(np.float32)

# df['description_feature'].apply(pd.Series)

### historical person

In [32]:
person_df = read_csv('historical_person')

{'name': 'historical_person'}
arg_name name=historical_person
__cachefile__name=historical_person
run


In [33]:
person_df.head()

,object_id,name
0,00133be3ff222c9b74b0,Staten-Generaal
1,00133be3ff222c9b74b0,"Álvarez de Toledo, Fernando (3e hertog van Alva)"
2,0026e030a0209b834b3e,Raad van Beroerten
3,0026e030a0209b834b3e,"Álvarez de Toledo, Fernando (3e hertog van Alva)"
4,00440ec5a4783b4b6bdb,Dolle Mina


In [34]:
data = pd.DataFrame({
    'a': [1, 2, 2, 2, 3, 3],
    'b': [0, 1, 0, 1, 1, 1]
})
data

,a,b
0,1,0
1,2,1
2,2,0
3,2,1
4,3,1
5,3,1


In [35]:
pd.crosstab(data['a'], data['b'])

b,0,1
a,,
1,1,0
2,1,2
3,0,2


In [36]:

# 出現回数30以上に絞る
vc = person_df['name'].value_counts()
use_names = vc[vc > 30].index

# isin で 30 回以上でてくるようなレコードに絞り込んでから corsstab を行なう
# 行数が多すぎるのを防ぐため
idx = person_df['name'].isin(use_names)
_use_df = person_df[idx].reset_index(drop=True)
pd.crosstab(_use_df['object_id'], _use_df['name'])

name,"Djatiroto, Suikeronderneming",Frederik Hendrik (prins van Oranje),Handels Vereeniging Amsterdam,Jacobus II (koning van Engeland en Schotland),Lodewijk XIV (koning van Frankrijk),"Maria II Stuart (koningin van Engeland, Schotland en Ierland)",Maurits (prins van Oranje),"Musschenbroek, Sam van","Oldenbarnevelt, Johan van","Titzenthaler, Eckart","Titzenthaler, Marba",Tjomal Suikerfabriek,Verenigde Oostindische Compagnie,Wehrmacht,Wilhelm II (keizer van Duitsland),Wilhelmina (koningin der Nederlanden),Willem I (prins van Oranje),Willem III (koning der Nederlanden),"Willem III (prins van Oranje en koning van Engeland, Schotland en Ierland)","Wirix, F.J.","Witt, Cornelis de","Witt, Johan de","Álvarez de Toledo, Fernando (3e hertog van Alva)"
object_id,,,,,,,,,,,,,,,,,,,,,,,
00133be3ff222c9b74b0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0026e030a0209b834b3e,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
009d44bd779a8722b00c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
00cd543929cbd97be573,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
00d601332278b5dad17e,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff4e08fb9e8887bbe1a3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ff657e8134633815bf1d,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
ff7b9c0a4a4cbaa7f7db,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
OBJECT_ID='object_id'
def left_join(left, right, on=OBJECT_ID):
    if isinstance(left, pd.DataFrame):
        left = left[on]
    return pd.merge(left, right, on=on, how='left').drop(columns=[on])

class HistoricalPersonBlock(AbstractBaseBlock):
    def __init__(self, min_count=30):
        self.min_count = min_count

    def fit(self, input_df, y=None):
        person_df = read_csv('historical_person')

        vc = person_df['name'].value_counts()
        # 出現回数30以上に絞る
        use_names = vc[vc >= self.min_count].index

        # isin で 30 回以上でてくるようなレコードに絞り込んでから corsstab を行なう
        idx = person_df['name'].isin(use_names)
        _use_df = person_df[idx].reset_index(drop=True)

        self.agg_df_ = pd.crosstab(_use_df['object_id'], _use_df['name'])
        return self.transform(input_df)

    def transform(self, input_df):
        out_df = left_join(input_df, self.agg_df_)
        return out_df.add_prefix(f'HistoricalPersonBlock_name=')


In [38]:
block = HistoricalPersonBlock()
assert len(block.fit(train)) == len(train)
assert block.transform(test).equals(block.transform(test))

{'name': 'historical_person'}
arg_name name=historical_person
__cachefile__name=historical_person


### production_place
* geopyを使ってproduction_placeを地名→国名に変換する
* https://www.guruguru.science/competitions/16/discussions/970ced6d-f974-4979-8f04-dbcf1c2f51a0/

In [39]:
!pip install geopy

In [40]:
from geopy.geocoders import Nominatim

def place2country(address):
    geolocator = Nominatim(user_agent='sample', timeout=200)
    loc = geolocator.geocode(address, language='en')
    coordinates = (loc.latitude, loc.longitude)
    location = geolocator.reverse(coordinates, language='en')
    country = location.raw['address']['country']
    return country

In [41]:
OBJECT_ID='object_id'
def left_join(left, right, on=OBJECT_ID):
    print(left.shape)
    if isinstance(left, pd.DataFrame):
        left = left[on]
    print(left.shape)
    return pd.merge(left, right, on=on, how='left').drop(columns=[on])

class ProductionPlaceBlock(AbstractBaseBlock):
    def fit(self, input_df, y=None):
        production_place_df = read_csv('production_place')
        place_list = production_place_df['name'].unique()
        self.country_dict = {}

        for place in place_list:
            try:
                country = place2country(place)
                self.country_dict[place] = country
            except:
                # 国名を取得できない場合は nan
                print('nan place', place)
                self.country_dict[place] = np.nan

        self.encord_dict = defaultdict(lambda: np.nan)
        for i, (k, v) in enumerate(self.country_dict.items()):
            self.encord_dict[v] = i
        return self.transform(input_df)

    def transform(self, input_df):
        production_place_df = read_csv('production_place')
        production_place_df['country_name'] = production_place_df['name'].map(self.country_dict)
        out_df = production_place_df.drop('name', axis=1).drop_duplicates(subset=['object_id'])
        out_df = left_join(input_df, out_df)
        out_df['country_name'] = out_df['country_name'].map(self.encord_dict)

        return out_df.add_prefix(f'ProductionPlaceBlock_')


In [42]:
block = ProductionPlaceBlock()
# print(len(train))
# print(len(block.fit(train)))
assert len(block.fit(train)) == len(train)
assert block.transform(test).equals(block.transform(test))


{'name': 'production_place'}
arg_name name=production_place
__cachefile__name=production_place
run
nan place Indonesian Archipelago
nan place Nisida, Isola di
{'name': 'production_place'}
arg_name name=production_place
__cachefile__name=production_place
(12026, 20)
(12026,)
{'name': 'production_place'}
arg_name name=production_place
__cachefile__name=production_place
(12008, 18)
(12008,)
{'name': 'production_place'}
arg_name name=production_place
__cachefile__name=production_place
(12008, 18)
(12008,)


### material

In [43]:
# material_df = read_csv('material')
# t =pd.crosstab(material_df['object_id'], material_df['name']).reset_index()
# print(t.shape)
# #t = t[t['object_id'].isin(train['object_id'])]
# print(train.shape)
# left_join(train, t).head()

In [44]:
class MaterialBlock(AbstractBaseBlock):
    def fit(self, input_df, y=None):
        material_df = read_csv('material')
        self.tab = pd.crosstab(material_df['object_id'], material_df['name']).reset_index()
        
        return self.transform(input_df)

    def transform(self, input_df):
        out_df = left_join(input_df, self.tab)
        return out_df.add_prefix(f'MaterialBlock_')

In [45]:
block = MaterialBlock()
assert len(block.fit(train)) == len(train)
assert block.transform(test).equals(block.transform(test))

{'name': 'material'}
arg_name name=material
__cachefile__name=material
run
(12026, 20)
(12026,)
(12008, 18)
(12008,)
(12008, 18)
(12008,)


### technique

In [46]:
# print(train.shape)
# technique_df = read_csv('technique')
# # print(technique_df.shape)
# # print(technique_df.object_id.unique().shape)
# g = technique_df.groupby('object_id').groups
# print(type(g))
# i = 0

# for objid, idx in g.items():
#      names = sorted(list(technique_df.iloc[idx]['name']))
#      name = ','.join(names) #複数技法がある場合は連結する
#      print(name)
#      i+=1
#      if i > 10:
#         break
# #left_join(train, technique_df).shape

In [47]:
class TechniqueBlock(AbstractBaseBlock):
    def fit(self, input_df, y=None):
        technique_df = read_csv('technique')
        self.tab = pd.crosstab(technique_df['object_id'], technique_df['name']).reset_index()
        
        return self.transform(input_df)

    def transform(self, input_df):
        out_df = left_join(input_df, self.tab)
        return out_df.add_prefix(f'TechniqueBlock_')

In [48]:
block = TechniqueBlock()
assert len(block.fit(train)) == len(train)
assert block.transform(test).equals(block.transform(test))

{'name': 'technique'}
arg_name name=technique
__cachefile__name=technique
run
(12026, 20)
(12026,)
(12008, 18)
(12008,)
(12008, 18)
(12008,)


### object_collection

In [49]:
class ObjectCollectionBlock(AbstractBaseBlock):
    def fit(self, input_df, y=None):
        object_collection_df = read_csv('object_collection')
        self.tab = pd.crosstab(object_collection_df['object_id'], object_collection_df['name']).reset_index()
        
        return self.transform(input_df)

    def transform(self, input_df):
        out_df = left_join(input_df, self.tab)
        return out_df.add_prefix(f'ObjectCollection_')

In [50]:
block = ObjectCollectionBlock()
assert len(block.fit(train)) == len(train)
assert block.transform(test).equals(block.transform(test))

{'name': 'object_collection'}
arg_name name=object_collection
__cachefile__name=object_collection
run
(12026, 20)
(12026,)
(12008, 18)
(12008,)
(12008, 18)
(12008,)


### palette

In [51]:
class PaletteBlock(AbstractBaseBlock):
    def transform(self, input_df):
        output_df = input_df.copy()
        palette = read_csv('palette')

        # ratio最大のものを取得
        max_palette = palette.groupby('object_id')['ratio'].max().reset_index()
        max_palette = pd.merge(max_palette, palette, on=['object_id','ratio'], how='left').rename(
            columns={"ratio":"max_ratio", "color_r":"max_palette_r", "color_g":"max_palette_g","color_b":"max_palette_b"})  
        max_palette = max_palette.loc[max_palette["object_id"].drop_duplicates().index.tolist()].reset_index()  # 同じidでmax ratioが同じものは削除
        output_df = pd.merge(output_df, max_palette, on="object_id", how="left")

        # 平均のrgbを取得
        mean_palette = palette.copy()
        mean_palette["color_r"] = palette["ratio"] * palette["color_r"]
        mean_palette["color_g"] = palette["ratio"] * palette["color_g"]
        mean_palette["color_b"] = palette["ratio"] * palette["color_b"]
        mean_palette = mean_palette.groupby("object_id").sum().reset_index().rename(
            columns={"color_r":"mean_palette_r", "color_g":"mean_palette_g","color_b":"mean_palette_b"})
        output_df = pd.merge(output_df, mean_palette, on="object_id", how="left")

        # 標準偏差でRGBのばらつきを取得
        var_palette = palette.copy()
        var_palette = var_palette.groupby("object_id").std().reset_index().rename(
            columns={"color_r":"std_palette_r", "color_g":"std_palette_g","color_b":"std_palette_b"})
        #print(var_palette.head())
        output_df = pd.merge(output_df, var_palette, on="object_id", how="left")

        use_columns = ["max_ratio", "max_palette_r","max_palette_g","max_palette_b",
                    "mean_palette_r","mean_palette_g","mean_palette_b",
                    "std_palette_r","std_palette_g","std_palette_b",
                    ]
        return output_df[use_columns]

In [52]:
block = PaletteBlock()
assert len(block.fit(train)) == len(train)
assert block.transform(test).equals(block.transform(test))

{'name': 'palette'}
arg_name name=palette
__cachefile__name=palette
run
{'name': 'palette'}
arg_name name=palette
__cachefile__name=palette
{'name': 'palette'}
arg_name name=palette
__cachefile__name=palette


### subtitle

In [53]:
# TODO: 他のテーブルも同じように追加
class SubtitleBlock(AbstractBaseBlock):
    def __init__(self):
        pass

    def transform(self, input_df):
        out_df = pd.DataFrame()
        for axis in ['h', 'w', 't', 'd']:
            column_name = f'size_{axis}'
            # 正規表現を使ってサイズを抽出
            size_info = input_df['sub_title'].str.extract(r'{} (\d*|\d*\.\d*)(cm|mm)'.format(axis)) 
            size_info = size_info.rename(columns={0: column_name, 1: 'unit'})
            # dtypeがobjectになってるのでfloatに直す
            size_info[column_name] = size_info[column_name].replace('', np.nan).astype(float) 
            size_info[column_name] = size_info.apply(lambda row: row[column_name] * 10 if row['unit'] == 'cm' else row[column_name], axis=1) # 　単位をmmに統一する
            out_df[column_name] = size_info[column_name] 

        return out_df.add_prefix(f'SubtitleBlock_')


In [54]:
block = SubtitleBlock()
assert len(block.fit(train)) == len(train)
assert block.transform(test).equals(block.transform(test))

### ベン図で分布がずれているか確認する
* train と test で分布がずれて否か調べる

In [55]:
def filter_names(master_df: pd.DataFrame, input_df: pd.DataFrame) -> set:
    idx = master_df['object_id'].isin(input_df['object_id'])
    return set(master_df[idx]['name'].unique())

In [56]:
fig, ax = plt.subplots(figsize=(6, 4))
venn2(subsets=(filter_names(person_df, train), 
               filter_names(person_df, test)), set_labels=('Train', 'Test'), ax=ax)
ax.set_title('Historical Person の分布')


Text(0.5, 1.0, 'Historical Person の分布')

In [57]:
# 一般化
def plot_one2many_relation_table_intersection(name: str, ax: plt.Axes) -> plt.Axes:
    master_df = read_csv(name)
    venn2(subsets=(filter_names(master_df, train), filter_names(master_df, test)), 
          set_labels=('Train', 'Test'), 
          ax=ax, 
          set_colors=('C0', 'C1'))
    ax.set_title(f'{name}の分布')
    return ax

In [58]:
fig, ax = plt.subplots()
plot_one2many_relation_table_intersection('production_place', ax)

{'name': 'production_place'}
arg_name name=production_place
__cachefile__name=production_place


In [59]:
table_names = [
    'production_place',
    'material',
    'object_collection',
    'historical_person',
    'technique'
]

In [60]:
n_figs = len(table_names)
n_cols = 3
n_rows = - (- n_figs // n_cols)

fig, axes = plt.subplots(figsize=(4 * n_cols, 3 * n_rows), ncols=n_cols, nrows=n_rows)
axes = np.ravel(axes)

for name, ax in zip(table_names, axes):
    plot_one2many_relation_table_intersection(name=name, ax=ax)

fig.tight_layout()

{'name': 'production_place'}
arg_name name=production_place
__cachefile__name=production_place
{'name': 'material'}
arg_name name=material
__cachefile__name=material
{'name': 'object_collection'}
arg_name name=object_collection
__cachefile__name=object_collection
{'name': 'historical_person'}
arg_name name=historical_person
__cachefile__name=historical_person
{'name': 'technique'}
arg_name name=technique
__cachefile__name=technique


In [61]:
!!pip install mlxtend

['Requirement already satisfied: mlxtend in /usr/local/lib/python3.7/dist-packages (0.14.0)',
 'Requirement already satisfied: numpy>=1.10.4 in /usr/local/lib/python3.7/dist-packages (from mlxtend) (1.19.5)',
 'Requirement already satisfied: matplotlib>=1.5.1 in /usr/local/lib/python3.7/dist-packages (from mlxtend) (3.2.2)',
 'Requirement already satisfied: setuptools in /usr/local/lib/python3.7/dist-packages (from mlxtend) (54.0.0)',
 'Requirement already satisfied: scipy>=0.17 in /usr/local/lib/python3.7/dist-packages (from mlxtend) (1.4.1)',
 'Requirement already satisfied: scikit-learn>=0.18 in /usr/local/lib/python3.7/dist-packages (from mlxtend) (0.22.2.post1)',
 'Requirement already satisfied: pandas>=0.17.1 in /usr/local/lib/python3.7/dist-packages (from mlxtend) (1.1.5)',
 'Requirement already satisfied: python-dateutil>=2.1 in /usr/local/lib/python3.7/dist-packages (from matplotlib>=1.5.1->mlxtend) (2.8.1)',
 'Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=

In [62]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# materialについて
target_df = read_csv('material')
df = pd.crosstab(target_df['object_id'], target_df['name'])
df

{'name': 'material'}
arg_name name=material
__cachefile__name=material


name,China paper,Japanese paper (handmade paper),RC paper,acrylic paint,alloy,amalgam,asbestos,asbestos-cement,ball-point pens,baryta paper,brass (alloy),bronze (metal),canvas,cardboard,casein paint,celluloid (cellulosic),chalk,copper (metal),cotton (textile),deck paint,enamel paint,gilding (material),glass,gold (metal),gold leaf,gouache (paint),hardboard,horn (animal material),human hair,ink,iron (metal),ivory,jade (rock),lacquer (coating),lead (metal),leather,linden,linen (material),looking glasses,mahogany (wood),metal,mother of pearl,oak (wood),oil (organic material),oil paint (paint),paint (coating),panel,paper,papier mâché,parchment (animal material),particle board,pear (wood),pencil,photographic paper,plush,plywood,poplar (wood),porcelain,prepared paper,printing ink,rope,silk,silver (metal),silver leaf,slate (rock),softwood,tempera,textile materials,tin (metal),tin alloy,tracing paper,transparant paper,velvet (fabric weave),watercolor (paint),wax,wood (plant material),wood board
object_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000405d9a5e3f49fc49d,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
001020bd00b149970f78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0011d6be41612ec9eae3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0012765f7a97ccc3e9e9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
00133be3ff222c9b74b0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff1d87d79953ddab2c6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
fff4bbb55fd7702d294e,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
fffbe07b997bec00e203,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
 #material ごとの 全オブジェクト中の割合. 複数持つobjectもあるので supportの和は1より大きい
freq_item_df = apriori(df, min_support=.005, use_colnames=True)
freq_item_df.sort_values('support', ascending=False)

,support,itemsets
9,0.561816,(paper)
10,0.322861,(photographic paper)
7,0.175146,(oil paint (paint))
2,0.172730,(cardboard)
14,0.153947,"(photographic paper, cardboard)"
20,0.104766,"(photographic paper, paper)"
1,0.094124,(canvas)
12,0.093445,"(oil paint (paint), canvas)"
8,0.071865,(panel)
19,0.069575,"(oil paint (paint), panel)"


In [64]:
association_rules(freq_item_df, metric='lift').sort_values('lift', ascending=False)
sns.clustermap(df.corr(), cmap='Blues')

## テキスト特徴
テキスト系データは分析には適さない文字や単語が含まれています。例えば文章の意味分析をする上では英語で言う前置詞やよくでてくる I, you といった単語はなんにでも出てくるため、文章の意味情報には大きな影響を与えないでしょう。こういった単語を取り除く作業を行ったり、記号や数字を取り除く、大文字小文字をすべて小文字にする、など文章を「機械から見た時に綺麗にする」処理を行なうことを「前処理」と言います。

とくに TFIDF のようにカウントベースの特徴は表記ゆれに弱く (活用形などで少しでも表現がずれると別のものとされてしまう)、また一般的に出てくるワードもカウントしてしまいノイズになりやすいですので、前処理は行ったほうが良いです。

自分で正規表現など書いても良いですがそれなりに大変なためライブラリの力を借りましょう。今回は texthero を紹介したいと思います。texthero は自然言語処理 (NLP) の前処理などを行なうためのライブラリです。

https://github.com/jbesomi/texthero



In [65]:
!pip install texthero

     |████████████████████████████████| 1.4MB 23.1MB/s 
     |████████████████████████████████| 245kB 45.4MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434676 sha256=9b8865b49ea3ae1bfc4eb428c12d3d781d91fc038a869ed0ace0eee79f065b56
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [66]:
import texthero as hero

raw_text = train['description']

clean_text = hero.clean(raw_text, pipeline=[
    hero.preprocessing.fillna,
    hero.preprocessing.lowercase,
    hero.preprocessing.remove_digits,
    hero.preprocessing.remove_punctuation,
    hero.preprocessing.remove_diacritics,
    hero.preprocessing.remove_stopwords
])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [67]:
# これがこうなる
i = 101
print('{} \n-> {}'.format(raw_text.values[i], clean_text.values[i]))

Berglandschap met een waterval tussen rotsen en boomstammen. De rivier stroomt door een dal tussen twee bergen waarop kastelen staan. 
-> berglandschap met een waterval tussen rotsen en boomstammen  de rivier stroomt door een dal tussen twee bergen waarop kastelen staan 


In [68]:
hero.visualization.wordcloud(clean_text, colormap='viridis', background_color='white')


なるほどよくわかりませんが、短い単語 (de / en など) がいくつもあってなんとなく前置詞とかの意味をなさない単語のように見えますね。discussion でもあがっていましたが text はオランダ語である場合があります。オランダ語での stopword も考慮するように少し直します。

In [69]:
import nltk

nltk.download('stopwords')
os.listdir(os.path.expanduser('~/nltk_data/corpora/stopwords/'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['kazakh',
 'arabic',
 'norwegian',
 'french',
 'italian',
 'finnish',
 'danish',
 'greek',
 'spanish',
 'dutch',
 'english',
 'hungarian',
 'azerbaijani',
 'german',
 'russian',
 'nepali',
 'slovene',
 'tajik',
 'swedish',
 'README',
 'indonesian',
 'turkish',
 'portuguese',
 'romanian']

In [70]:
# 英語とオランダ語を stopword として指定
custom_stopwords = nltk.corpus.stopwords.words('dutch') + nltk.corpus.stopwords.words('english')

apply_stopword_text = hero.clean(raw_text, pipeline=[
    hero.preprocessing.fillna,
    hero.preprocessing.lowercase,
    hero.preprocessing.remove_digits,
    hero.preprocessing.remove_punctuation,
    hero.preprocessing.remove_diacritics,
    lambda x: hero.preprocessing.remove_stopwords(x, stopwords=custom_stopwords)
])

In [71]:
hero.visualization.wordcloud(apply_stopword_text, colormap='viridis', background_color='white')

###  特徴量へ変換 TF-IDF

In [78]:
apply_stopword_text[2]

'portret  eenvrouw  zittend naast  tafel '

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=100000)),
    ('svd', TruncatedSVD(n_components=50)),
])
z = pipeline.fit_transform(apply_stopword_text)

In [73]:
z.shape

(12026, 50)

In [1]:
mat_col = pd.concat([material, collection], axis=0).reset_index(drop=True)
mat_tec = pd.concat([material, technique], axis=0).reset_index(drop=True)
col_tec = pd.concat([collection, technique], axis=0).reset_index(drop=True)
mat_col_tec = pd.concat([material, collection, technique], axis=0).reset_index(drop=True)

NameError: ignored

### TSNEによる可視化
* sklearn にもあるが遅い

In [ ]:
!pip install MulticoreTSNE

  Created wheel for MulticoreTSNE: filename=MulticoreTSNE-0.1-cp37-cp37m-linux_x86_64.whl size=68502 sha256=a048ea7836d5c1cfbcf151cd843dd6f04ba477df3045d29452219c57989702c8
  Stored in directory: /root/.cache/pip/wheels/27/59/53/3b52ee63add3692254c30d687fa4dff4d128d0557861fb028e
Successfully built MulticoreTSNE


In [ ]:
from MulticoreTSNE import MulticoreTSNE as TSNE

def show_tsne():
    # tsneおそいのでちょっと時間かかる
    with Timer(prefix='run tsne'):
        tsne = TSNE(n_jobs=-1)
        embedding = tsne.fit_transform(z)

    # わかりやすさのため１０を亭としたlog変換
    train['log_likes'] = np.log10(train['likes'] + 1)

    bin_labels = pd.cut(train['log_likes'], bins=5)
    fig, ax = plt.subplots(figsize=(10, 10))

    ax.scatter(*embedding.T, c=bin_labels.cat.codes, s=20, alpha=.8, cmap='cividis')
    ax.grid()

#show_tsne()

In [82]:
# わかりやすさのため１０を亭としたlog変換
train['log_likes'] = np.log10(train['likes'] + 1)

bin_labels = pd.cut(train['log_likes'], bins=5)
print(bin_labels)
bin_labels.cat.codes # 整数label化したものを取得可能

0           (0.888, 1.775]
1        (-0.00444, 0.888]
2        (-0.00444, 0.888]
3           (1.775, 2.663]
4           (0.888, 1.775]
               ...        
12021    (-0.00444, 0.888]
12022    (-0.00444, 0.888]
12023    (-0.00444, 0.888]
12024       (0.888, 1.775]
12025       (1.775, 2.663]
Name: log_likes, Length: 12026, dtype: category
Categories (5, interval[float64]): [(-0.00444, 0.888] < (0.888, 1.775] < (1.775, 2.663] < (2.663, 3.55] <
                                    (3.55, 4.438]]


0        1
1        0
2        0
3        2
4        1
        ..
12021    0
12022    0
12023    0
12024    1
12025    2
Length: 12026, dtype: int8

色が明るいほどlikeが大きいもの

### TF-idf Block

In [ ]:
def text_normalization(text):

    # 英語とオランダ語を stopword として指定
    custom_stopwords = nltk.corpus.stopwords.words('dutch') + nltk.corpus.stopwords.words('english')

    x = hero.clean(text, pipeline=[
        hero.preprocessing.fillna,
        hero.preprocessing.lowercase,
        hero.preprocessing.remove_digits,
        hero.preprocessing.remove_punctuation,
        hero.preprocessing.remove_diacritics,
        lambda x: hero.preprocessing.remove_stopwords(x, stopwords=custom_stopwords)
    ])

    return x

class TfidfBlock(AbstractBaseBlock):
    """tfidf x SVD による圧縮を行なう block"""
    def __init__(self, column: str):
        """
        args:
            column: str
                変換対象のカラム名
        """
        self.column = column

    def preprocess(self, input_df):
        x = text_normalization(input_df[self.column])
        return x

    def get_master(self, input_df):
        """tdidfを計算するための全体集合を返す. 
        デフォルトでは fit でわたされた dataframe を使うが, もっと別のデータを使うのも考えられる."""
        return input_df

    def fit(self, input_df, y=None):
        master_df = self.get_master(input_df)
        text = self.preprocess(input_df)
        self.pileline_ = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=10000)),
            ('svd', TruncatedSVD(n_components=50)),
        ])

        self.pileline_.fit(text)
        return self.transform(input_df)

    def transform(self, input_df):
        text = self.preprocess(input_df)
        z = self.pileline_.transform(text)

        out_df = pd.DataFrame(z)
        return out_df.add_prefix(f'{self.column}_tfidf_')

In [ ]:
block = TfidfBlock('title')

In [ ]:
block.fit(train)
assert block.transform(train).equals(block.transform(train))
assert block.transform(test).equals(block.transform(test))

### 言語判定特徴
* fasttextを使う

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
!pip install fastText
!rm -rf fastText
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 6.90 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
     |████████████████████████████████| 71kB 3.3MB/s 
  Created wheel for fastText: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3093693 sha256=546e0b04badeaefeca02e72a301e6eba86588f48792d099e111052ff723ad4c5
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fastText
--2021-03-13 07:47:01--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Savin

In [ ]:
from fasttext import load_model

# パスを書き換えてください
model = load_model("./lid.176.bin")
model.predict("Arcadian Landscape with Shepherds and Cattle")
model.predict("De schilder H.W. Mesdag voor een doek")

(('__label__nl',), array([1.0000515]))

In [ ]:
from fasttext import load_model
class ColumnLangBlock(AbstractBaseBlock):
    def __init__(self, column):
        self.column = column

    def fit(self, input_df, y=None):
        fs_model = load_model("./lid.176.bin")
        lang_list = list(set(input_df[self.column].fillna("").map(
            lambda x: fs_model.predict(x.replace("\n", ""))[0][0])))
        print(lang_list)
        self.encorder = defaultdict(lambda : np.nan)
        for i, lang in enumerate(lang_list):
            self.encorder[lang] = i
        print(self.encorder)
        return self.transform(input_df)

    def transform(self, input_df):
        fs_model = load_model("./lid.176.bin")
        out_df = pd.DataFrame()
        out_df[f"{self.column}_lang_ft"] = input_df[self.column].fillna("").map(
            lambda x: fs_model.predict(x.replace("\n", ""))[0][0])
        out_df[f"{self.column}_lang_ft"] = out_df[f"{self.column}_lang_ft"].map(self.encorder)
        return out_df

In [ ]:
block = ColumnLangBlock('title')
print(block)


In [ ]:
block.fit(train)
assert block.transform(train).equals(block.transform(train))
assert block.transform(test).equals(block.transform(test))

['__label__no', '__label__pt', '__label__sk', '__label__fa', '__label__id', '__label__ceb', '__label__en', '__label__eu', '__label__sq', '__label__hu', '__label__da', '__label__af', '__label__eo', '__label__de', '__label__az', '__label__li', '__label__ca', '__label__pl', '__label__br', '__label__it', '__label__fr', '__label__ms', '__label__la', '__label__es', '__label__hr', '__label__sv', '__label__fy', '__label__sl', '__label__nl', '__label__ia', '__label__fi', '__label__ru', '__label__et', '__label__tr']
defaultdict(<function ColumnLangBlock.fit.<locals>.<lambda> at 0x7f64056ac5f0>, {'__label__no': 0, '__label__pt': 1, '__label__sk': 2, '__label__fa': 3, '__label__id': 4, '__label__ceb': 5, '__label__en': 6, '__label__eu': 7, '__label__sq': 8, '__label__hu': 9, '__label__da': 10, '__label__af': 11, '__label__eo': 12, '__label__de': 13, '__label__az': 14, '__label__li': 15, '__label__ca': 16, '__label__pl': 17, '__label__br': 18, '__label__it': 19, '__label__fr': 20, '__label__ms': 21

### word2vec

## 作った特徴量を定義する

In [ ]:
feature_blocks = [
    # one hot encoding
    *[OneHotEncoding(c) for c in [
        'title', 'long_title',
       'principal_maker', 'principal_or_first_maker', 'sub_title',
       'copyright_holder', 'more_title', 'acquisition_method',
       'acquisition_date', 'acquisition_credit_line', 'dating_presenting_date',
       'dating_sorting_date', 'dating_period', 'dating_year_early',
       'dating_year_late',]],

    # count encoding
    *[CountEncodingBlock(c) for c in [
        'art_series_id', 
        'title', 'description', 'long_title',
       'principal_maker', 'principal_or_first_maker', 'sub_title',
       'copyright_holder', 'more_title', 'acquisition_method',
       'acquisition_date', 'acquisition_credit_line', 'dating_presenting_date',
       'dating_sorting_date', 'dating_period', 'dating_year_early',
       'dating_year_late',]],

    # 文字列長さ
    *[StringLengthBlock(c) for c in [
        'title', 'description', 'long_title',
       'principal_maker', 'principal_or_first_maker', 'sub_title',
    ]],
    # historical person
    HistoricalPersonBlock(),

    # description の tfidf
    TfidfBlock('description'),

    # タイトル言語特徴
    ColumnLangBlock('title'),

    # BERT
    #BertEmbeddingBlock('description', umap_components=50),
    # material
    MaterialBlock(),
    # technique
    TechniqueBlock(),
    # object_collection 
    ObjectCollectionBlock(),
    # palette
    PaletteBlock(),
    # 作成された国名
    ProductionPlaceBlock(), 
    # subtitle
    SubtitleBlock()

]

train_feat_df = run_blocks(train, blocks=feature_blocks, test=False)

★★★★★★★★★★★★★★★★★★★★ start run blocks... ★★★★★★★★★★★★★★★★★★★★
	- <__main__.OneHotEncoding object at 0x7f6405406250> 0.012[s]
	- <__main__.OneHotEncoding object at 0x7f6405406490> 0.008[s]
	- <__main__.OneHotEncoding object at 0x7f64053feb50> 0.007[s]
	- <__main__.OneHotEncoding object at 0x7f64053fec10> 0.007[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe810> 0.008[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe2d0> 0.005[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe650> 0.008[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe490> 0.004[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe050> 0.006[s]
	- <__main__.OneHotEncoding object at 0x7f64053fea50> 0.005[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe390> 0.007[s]
	- <__main__.OneHotEncoding object at 0x7f64053fed90> 0.007[s]
	- <__main__.OneHotEncoding object at 0x7f64053fee90> 0.003[s]
	- <__main__.OneHotEncoding object at 0x7f64053fef90> 0.007[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe290> 0.

['__label__no', '__label__pt', '__label__sk', '__label__fa', '__label__id', '__label__ceb', '__label__en', '__label__eu', '__label__sq', '__label__hu', '__label__da', '__label__af', '__label__eo', '__label__de', '__label__az', '__label__li', '__label__ca', '__label__pl', '__label__br', '__label__it', '__label__fr', '__label__ms', '__label__la', '__label__es', '__label__hr', '__label__sv', '__label__fy', '__label__sl', '__label__nl', '__label__ia', '__label__fi', '__label__ru', '__label__et', '__label__tr']
defaultdict(<function ColumnLangBlock.fit.<locals>.<lambda> at 0x7f6402fedef0>, {'__label__no': 0, '__label__pt': 1, '__label__sk': 2, '__label__fa': 3, '__label__id': 4, '__label__ceb': 5, '__label__en': 6, '__label__eu': 7, '__label__sq': 8, '__label__hu': 9, '__label__da': 10, '__label__af': 11, '__label__eo': 12, '__label__de': 13, '__label__az': 14, '__label__li': 15, '__label__ca': 16, '__label__pl': 17, '__label__br': 18, '__label__it': 19, '__label__fr': 20, '__label__ms': 21

	- <__main__.ColumnLangBlock object at 0x7f64053fec50> 0.670[s]
{'name': 'material'}
arg_name name=material
__cachefile__name=material
(12026, 20)
(12026,)
	- <__main__.MaterialBlock object at 0x7f64053fee10> 0.380[s]
{'name': 'technique'}
arg_name name=technique
__cachefile__name=technique
(12026, 20)
(12026,)
	- <__main__.TechniqueBlock object at 0x7f64053fef10> 0.199[s]
{'name': 'object_collection'}
arg_name name=object_collection
__cachefile__name=object_collection
(12026, 20)
(12026,)
	- <__main__.ObjectCollectionBlock object at 0x7f64053fe5d0> 0.121[s]
{'name': 'palette'}
arg_name name=palette
__cachefile__name=palette
	- <__main__.PaletteBlock object at 0x7f64053fe9d0> 0.629[s]
{'name': 'production_place'}
arg_name name=production_place
__cachefile__name=production_place
nan place Indonesian Archipelago
nan place Nisida, Isola di
{'name': 'production_place'}
arg_name name=production_place
__cachefile__name=production_place
(12026, 20)
(12026,)
	- <__main__.ProductionPlaceBlock o

In [ ]:
train_feat_df.head()

,title=Portret van een onbekende vrouw@OneHotEncoding,title=Portret van een onbekende man@OneHotEncoding,principal_maker=anonymous@OneHotEncoding,principal_maker=Johann Sadeler (I)@OneHotEncoding,principal_maker=Bernard Picart@OneHotEncoding,principal_maker=Richard Tepe@OneHotEncoding,principal_maker=anoniem (Monumentenzorg)@OneHotEncoding,principal_maker=Jan Luyken@OneHotEncoding,principal_maker=Raphaël Sadeler (I)@OneHotEncoding,principal_maker=Woodbury & Page@OneHotEncoding,principal_maker=Romeyn de Hooghe@OneHotEncoding,principal_maker=Aegidius Sadeler@OneHotEncoding,principal_maker=Giorgio Sommer@OneHotEncoding,principal_maker=Willem Diepraam@OneHotEncoding,principal_maker=Jacob Olie jr.@OneHotEncoding,principal_maker=Geldolph Adriaan Kessler@OneHotEncoding,principal_maker=Abraham Rademaker@OneHotEncoding,principal_maker=L. Heldring@OneHotEncoding,principal_maker=Hermanus Jan Hendrik van Rijkelijkhuysen@OneHotEncoding,principal_maker=Jan Harmensz. Muller@OneHotEncoding,principal_maker=Crispijn van de Passe (I)@OneHotEncoding,principal_maker=Johanna Margaretha Piek@OneHotEncoding,principal_maker=Cornelis Galle (I)@OneHotEncoding,principal_maker=Jacob de Gheyn (II)@OneHotEncoding,principal_maker=Cornelis Ploos van Amstel@OneHotEncoding,principal_maker=Théodore van Lelyveld@OneHotEncoding,principal_maker=Jacob van der Schley@OneHotEncoding,principal_maker=Daniël Veelwaard (I)@OneHotEncoding,principal_maker=Jan Frederik Christiaan Reckleben@OneHotEncoding,principal_maker=Paulus Pontius@OneHotEncoding,principal_maker=Albert Greiner@OneHotEncoding,principal_maker=Hendrick Goltzius@OneHotEncoding,principal_maker=Francis Frith@OneHotEncoding,principal_maker=Henry Pauw van Wieldrecht@OneHotEncoding,principal_maker=Heinrich Aldegrever@OneHotEncoding,principal_maker=Theodoor Koning@OneHotEncoding,principal_maker=Roelant Roghman@OneHotEncoding,principal_maker=Adolphe Burdet@OneHotEncoding,principal_maker=Jacques Lalaing@OneHotEncoding,principal_maker=G. Dangereux@OneHotEncoding,...,TechniqueBlock_inkjet print@TechniqueBlock,TechniqueBlock_lacquering@TechniqueBlock,TechniqueBlock_letterpress printing@TechniqueBlock,TechniqueBlock_litho etching@TechniqueBlock,TechniqueBlock_oil transfer print@TechniqueBlock,TechniqueBlock_painting@TechniqueBlock,TechniqueBlock_pen@TechniqueBlock,TechniqueBlock_perforating@TechniqueBlock,TechniqueBlock_photolithography@TechniqueBlock,TechniqueBlock_printed matter@TechniqueBlock,TechniqueBlock_printing@TechniqueBlock,TechniqueBlock_salted paper print@TechniqueBlock,TechniqueBlock_slide@TechniqueBlock,TechniqueBlock_steel engraving@TechniqueBlock,TechniqueBlock_writing (processes)@TechniqueBlock,ObjectCollection_Navy Model Room@ObjectCollectionBlock,ObjectCollection_dollhouse@ObjectCollectionBlock,ObjectCollection_drawings@ObjectCollectionBlock,ObjectCollection_glass@ObjectCollectionBlock,ObjectCollection_jewellery@ObjectCollectionBlock,ObjectCollection_lace@ObjectCollectionBlock,ObjectCollection_musical instruments@ObjectCollectionBlock,ObjectCollection_paintings@ObjectCollectionBlock,ObjectCollection_paper@ObjectCollectionBlock,ObjectCollection_prints@ObjectCollectionBlock,max_ratio@PaletteBlock,max_palette_r@PaletteBlock,max_palette_g@PaletteBlock,max_palette_b@PaletteBlock,mean_palette_r@PaletteBlock,mean_palette_g@PaletteBlock,mean_palette_b@PaletteBlock,std_palette_r@PaletteBlock,std_palette_g@PaletteBlock,std_palette_b@PaletteBlock,ProductionPlaceBlock_country_name@ProductionPlaceBlock,SubtitleBlock_size_h@SubtitleBlock,SubtitleBlock_size_w@SubtitleBlock,SubtitleBlock_size_t@SubtitleBlock,SubtitleBlock_size_d@SubtitleBlock
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.108754,54.0,30.0,5.0,80.760794,52.411413,10.170802,41.500978,34.432694,16.243813,138,665.0,537.0,25.0,47.0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0

# 学習

In [ ]:
! pip install optuna > /dev/null

In [ ]:
import lightgbm as lgbm
#import optuna.integration.lightgbm as lgb
from sklearn.metrics import mean_squared_error
from typing import Union, Optional, Iterable

def fit_lgbm(X: np.ndarray, 
             y: np.ndarray, 
             cv: Iterable, 
             y_max: Optional[float] = None,
             params: Optional[dict] = None, 
             verbose: Optional[int] =50,
             #tuning Optional[bool]: False
             ):
    """lightGBM を CrossValidation の枠組みで学習を行なう function"""

    # パラメータがないときは、空の dict で置き換える
    if params is None:
        params = {}
    is_silent = verbose is None
    is_cut_y = y_max is not None

    models = []
    # training data の target と同じだけのゼロ配列を用意
    oof_pred = np.zeros_like(y, dtype=np.float)
    for i, (idx_train, idx_valid) in enumerate(cv): 
        # この部分が交差検証のところです。データセットを cv instance によって分割します
        # training data を trian/valid に分割
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        # 最大値が決まっている場合, 学習するデータだけ丸めを行なう
        if is_cut_y:
            idx_use = y_train < y_max
            y_train = np.where(idx_use, y_train, y_max) #最大値以上の箇所は最大値にする
        clf = lgbm.LGBMRegressor(**params)

        with Timer(prefix='fit fold={} '.format(i), verbose=verbose):
            clf.fit(x_train, y_train, 
                    eval_set=[(x_valid, y_valid)],  
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)

        if not is_silent:
            print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')

    if not is_silent:
        score = mean_squared_error(y, oof_pred) ** .5
        print('Whole RMSLE: {:.4f}'.format(score))
    return oof_pred, models

In [ ]:
params = {
    # 目的関数. これの意味で最小となるようなパラメータを探します. 
    'objective': 'rmse', 

     # 学習率. 小さいほどなめらかな決定境界が作られて性能向上に繋がる場合が多いです、
    # がそれだけ木を作るため学習に時間がかかります
    'learning_rate': .1,

    # L2 Reguralization
    'reg_lambda': 1.,
    # こちらは L1 
    'reg_alpha': .1,

    # 木の深さ. 深い木を許容するほどより複雑な交互作用を考慮するようになります
    'max_depth': 5, 

    # 木の最大数. early_stopping という枠組みで木の数は制御されるようにしていますのでとても大きい値を指定しておきます.
    'n_estimators': 10000, 

    # 木を作る際に考慮する特徴量の割合. 1以下を指定すると特徴をランダムに欠落させます。小さくすることで, まんべんなく特徴を使うという効果があります.
    'colsample_bytree': .5, 

    # 最小分割でのデータ数. 小さいとより細かい粒度の分割方法を許容します.
    'min_child_samples': 10,

    # bagging の頻度と割合
    'subsample_freq': 3,
    'subsample': .9,

    # 特徴重要度計算のロジック(後述)
    'importance_type': 'gain', 
    'random_state': 71,
}

In [ ]:
y = train['likes'].values
y = np.log1p(y)
from sklearn.model_selection import KFold

fold = KFold(n_splits=5, shuffle=True, random_state=71)
cv = list(fold.split(train_feat_df, y)) # もともとが generator なため明示的に list に変換する

print(decorate('start training'))
oof, models = fit_lgbm(train_feat_df.values, y, cv, y_max=7.0, params=params, verbose=500)
print(decorate('FINISHED'))

★★★★★★★★★★★★★★★★★★★★ start training ★★★★★★★★★★★★★★★★★★★★
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[399]	valid_0's rmse: 1.05028
fit fold=0  4.375[s]
Fold 0 RMSLE: 1.0503
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[337]	valid_0's rmse: 1.05235
fit fold=1  3.836[s]
Fold 1 RMSLE: 1.0524
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[279]	valid_0's rmse: 1.03643
fit fold=2  3.461[s]
Fold 2 RMSLE: 1.0364
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[393]	valid_0's rmse: 1.03151
fit fold=3  4.227[s]
Fold 3 RMSLE: 1.0315
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.01934
Early stopping, best iteration is:
[479]	valid_0's rmse: 1.01881
fit fold=4  4.982[s]
Fold 4 RMSLE: 1.0188
Whole RMSLE: 1.0380
★★★★★★★★★★★★★★★★★★★★ FINISHED ★★★★★★★★★★★★★

# 外れ値を処理したときの性能について考える
y の最大値を指定してそれ以上の値をすべて切り捨てする方法が有効であるかを試す



In [ ]:
out_of_folds = {}
for y_max in [5., 5.5, 6., 6.5, 7., None]:
    print(decorate(y_max))
    oof, _ = fit_lgbm(train_feat_df.values, y, cv, y_max=y_max, params=params, verbose=500)
    out_of_folds[y_max] = oof

★★★★★★★★★★★★★★★★★★★★ 5.0 ★★★★★★★★★★★★★★★★★★★★
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.07817
Early stopping, best iteration is:
[411]	valid_0's rmse: 1.07527
fit fold=0  4.349[s]
Fold 0 RMSLE: 1.0753
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.07045
Early stopping, best iteration is:
[469]	valid_0's rmse: 1.0693
fit fold=1  4.756[s]
Fold 1 RMSLE: 1.0693
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.04462
Early stopping, best iteration is:
[591]	valid_0's rmse: 1.04282
fit fold=2  5.757[s]
Fold 2 RMSLE: 1.0428
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.05936
Early stopping, best iteration is:
[414]	valid_0's rmse: 1.05839
fit fold=3  4.330[s]
Fold 3 RMSLE: 1.0584
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.03931
Early stopping, best iteration is:
[563]	valid_0's rmse: 1.039

* 外れ値はカットしたほうが性能良さげ.
* 閾値は 6.5-7くらい

In [ ]:
scores = []

for key, value in out_of_folds.items():
    score_i = mean_squared_error(y, value) ** .5
    scores += [[key, score_i]]

In [ ]:
def visualize_importance(models, feat_train_df):
    """lightGBM の model 配列の feature importance を plot する
    CVごとのブレを boxen plot として表現します.

    args:
        models:
            List of lightGBM models
        feat_train_df:
            学習時に使った DataFrame
    """
    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x='feature_importance', 
                  y='column', 
                  order=order, 
                  ax=ax, 
                  palette='viridis', 
                  orient='h')
    ax.tick_params(axis='x', rotation=90)
    ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    return fig, ax

fig, ax = visualize_importance(models, train_feat_df)

In [ ]:
test_feat_df = run_blocks(input_df=test, blocks=feature_blocks, test=True)

★★★★★★★★★★★★★★★★★★★★ start run blocks... ★★★★★★★★★★★★★★★★★★★★
	- <__main__.OneHotEncoding object at 0x7f6405406250> 0.004[s]
	- <__main__.OneHotEncoding object at 0x7f6405406490> 0.004[s]
	- <__main__.OneHotEncoding object at 0x7f64053feb50> 0.005[s]
	- <__main__.OneHotEncoding object at 0x7f64053fec10> 0.004[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe810> 0.003[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe2d0> 0.003[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe650> 0.003[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe490> 0.002[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe050> 0.004[s]
	- <__main__.OneHotEncoding object at 0x7f64053fea50> 0.004[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe390> 0.007[s]
	- <__main__.OneHotEncoding object at 0x7f64053fed90> 0.006[s]
	- <__main__.OneHotEncoding object at 0x7f64053fee90> 0.001[s]
	- <__main__.OneHotEncoding object at 0x7f64053fef90> 0.006[s]
	- <__main__.OneHotEncoding object at 0x7f64053fe290> 0.

	- <__main__.ColumnLangBlock object at 0x7f64053fec50> 0.333[s]
(12008, 18)
(12008,)
	- <__main__.MaterialBlock object at 0x7f64053fee10> 0.027[s]
(12008, 18)
(12008,)
	- <__main__.TechniqueBlock object at 0x7f64053fef10> 0.019[s]
(12008, 18)
(12008,)
	- <__main__.ObjectCollectionBlock object at 0x7f64053fe5d0> 0.015[s]
{'name': 'palette'}
arg_name name=palette
__cachefile__name=palette
	- <__main__.PaletteBlock object at 0x7f64053fe9d0> 0.595[s]
{'name': 'production_place'}
arg_name name=production_place
__cachefile__name=production_place
(12008, 18)
(12008,)
	- <__main__.ProductionPlaceBlock object at 0x7f64053fe850> 0.028[s]
	- <__main__.SubtitleBlock object at 0x7f64053fe090> 0.625[s]
run test=True 4.482[s]


In [ ]:
def revert_to_real(y_log):
    _pred = np.expm1(y_log)
    _pred = np.where(_pred < 0, 0, _pred)
    return _pred

pred = np.array([model.predict(test_feat_df.values) for model in models])
pred = np.mean(pred, axis=0)
pred = revert_to_real(pred)
sub_df = pd.DataFrame({ 'likes': pred })
sub_df.to_csv(os.path.join(OUTPUT_DIR, '0002__submission.csv'), index=False)

In [ ]:

fig, ax = plt.subplots(figsize=(8, 8))
sns.histplot(np.log1p(pred), label='Test Predict', ax=ax, color='black')
sns.histplot(oof, label='Out Of Fold', ax=ax, color='C1')
ax.legend()
ax.grid()